In [4]:
import torch
import torch.nn as nn

class TemporalConvModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv1d(1024, 256, kernel_size=3, padding=1),  # net.0
            nn.ReLU(),                                       # net.1
            nn.Dropout(0.3),                                 # net.2  ← capa extra
            nn.Conv1d(256, 128, kernel_size=3, padding=1),   # net.3
            nn.ReLU()                                        # net.4
        )
        self.fc = nn.Linear(128, 2)

    def forward(self, x):
        x = self.net(x)
        x = x.mean(dim=-1)  # pooling temporal
        return self.fc(x)


model = TemporalConvModel()
state = torch.load("checkpoints/temporal_conv.pt", map_location="cpu")
model.load_state_dict(state)
print("✅ Pesos cargados correctamente")



✅ Pesos cargados correctamente


In [2]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu

Looking in indexes: https://download.pytorch.org/whl/cpu
   ---------------------------------------- 0.0/109.2 MB ? eta -:--:--
   ---------------------------------------- 0.0/109.2 MB ? eta -:--:--
   ---------------------------------------- 0.3/109.2 MB ? eta -:--:--
   ---------------------------------------- 0.5/109.2 MB 1.2 MB/s eta 0:01:30
   ---------------------------------------- 0.8/109.2 MB 1.2 MB/s eta 0:01:34
   ---------------------------------------- 1.0/109.2 MB 1.3 MB/s eta 0:01:24
   ---------------------------------------- 1.3/109.2 MB 1.4 MB/s eta 0:01:20
    --------------------------------------- 1.6/109.2 MB 1.3 MB/s eta 0:01:21
    --------------------------------------- 1.8/109.2 MB 1.3 MB/s eta 0:01:23
    --------------------------------------- 2.1/109.2 MB 1.3 MB/s eta 0:01:20
    --------------------------------------- 2.4/109.2 MB 1.4 MB/s eta 0:01:19
    --------------------------------------- 2.6/109.2 MB 1.4 MB/s eta 0:01:19
    ------------------------